In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary

In [2]:
#loads data created from "ecoli base data"
datafile = "strain_state_cli.csv"
strain_state_cli = pd.read_csv(datafile)

In [3]:
strains = strain_state_cli['Strain']
strains = strains.unique()

In [4]:
strains

array(['O157:H7', 'O157:NM', '(H-)', 'O26', 'O126', 'O121', 'O45:H2',
       'O157', 'O103', 'O111', 'other', 'Typhimurium', 'unknown',
       'O146:H21', 'O145', 'O6:H16', 'O45', 'O111:NM', 'O26:NM', 'O84',
       'O121:H19', 'O26:H11', 'O27', 'O118:H16', 'O169:H41', 'I',
       '4,[5],12:i:-', 'Newport', 'GI.P7', '-', 'GI.7', 'Enteritidis',
       'GII.4', 'New', 'Orleans', 'Muenchen', nan, 'GI.P6', 'GI.6',
       'IIaA15G2R2', 'Other', 'O91:H14', 'O103:H2', 'O145:NM', '41',
       'O124', 'subgroup', 'D', 'O6', 'Sydney', 'O84:NM'], dtype=object)

In [5]:
driver = webdriver.Chrome()

In [6]:
base_url = 'https://www.ncbi.nlm.nih.gov/genome/browse/#!/prokaryotes/'

In [7]:
#DO NOT RUN UNLESS YOU WANT TO GET RID OF STRAINS ALREADY FOUND!
strain_df = pd.DataFrame()

In [8]:
strain_df.head()

""


In [9]:
strains_left = strains
strains_checked = []
rows_list = []

In [10]:
try:
    driver.close()
except:
    None

driver = webdriver.Chrome()
driver.get("https://www.ncbi.nlm.nih.gov/genome/browse/#!/prokaryotes/")
# originally had a list and removed the strain from the list after success, abandoned for 
# while len(strains_left) > 0:
for strain in strains_left:
    if strain in strains_checked:
        None
    else:
        #print("strains_left: ",len(strains_left - strains_checked))

        search_term = "escherichia coli " + strain
        search_url = base_url+search_term.replace(" ", "%20")
    #         this is all code to manipulate the search bar, abandoned for manipulating the url directly
    #         searchbox = driver.find_element_by_class_name('form-control')
    #         driver.get(search_url)
    #         print(search_term)
    #         time.sleep(5)
    #         #searchbox.clear()
    #         searchbox.send_keys(search_term, Keys.RETURN)
    #         soup = BeautifulSoup(driver.page_source, 'html.parser')
    #         soup.prettify()[1:100]

        control = True
        counter = 0

        while control and counter < 1:
            try:
                driver.get(search_url)
                print(search_term)
                #change pause time here
                time.sleep(5)

                soup = BeautifulSoup(driver.page_source, 'html.parser')

                first_name = [x for x in soup.find("table",class_='TableGrid').find_all('tr')]
                print("found row")

                row = {}
                for i,j in enumerate([x.text for x in first_name[1].find_all('td')]):
                    row[i] = j

                row['Search'] = strain
                rows_list.append(row)
                print("added to df")

                control = False
                strains_checked.append(strain)
    #           strains_left.remove(strain)

            except:
                print("scrape error: ",counter)
                counter+=1

escherichia coli O157:H7
found row
added to df
escherichia coli O157:NM
found row
added to df
escherichia coli (H-)
found row
scrape error:  0
escherichia coli O26
found row
added to df
escherichia coli O126
found row
scrape error:  0
escherichia coli O121
found row
added to df
escherichia coli O45:H2
found row
added to df
escherichia coli O157
found row
added to df
escherichia coli O103
found row
added to df
escherichia coli O111
found row
added to df
escherichia coli other
found row
scrape error:  0
escherichia coli Typhimurium
found row
scrape error:  0
escherichia coli unknown
found row
scrape error:  0
escherichia coli O146:H21
found row
added to df
escherichia coli O145
found row
scrape error:  0
escherichia coli O6:H16
found row
added to df
escherichia coli O45
found row
scrape error:  0
escherichia coli O111:NM
found row
added to df
escherichia coli O26:NM
found row
scrape error:  0
escherichia coli O84
found row
scrape error:  0
escherichia coli O121:H19
found row
scrape error

TypeError: must be str, not float

In [11]:
strain_df = pd.DataFrame(rows_list)
strain_df = strain_df.loc[:,['Search', 8, 9, 13, 
                      16, 21]]
strain_df = strain_df.rename(index=str, columns={8: "Length of Genome", 9: "%GC", 
                                     13: "Protein Coding Genes", 16: "Genes Overall", 21: "tRNA Genes"})

In [12]:
all_terms = pd.merge(strain_state_cli, strain_df, left_on='Strain', right_on='Search')
all_terms = all_terms.drop('Search', axis=1)

In [13]:
all_terms.head()

,Unnamed: 0,Year,Month,State,Primary Mode,Illnesses,Hospitalizations,Deaths,Outbreak ID,Strain,...,Landarea(mi2),HDI (2017)[note 1],HDI (1995),Average High (F),Average Low (F),Length of Genome,%GC,Protein Coding Genes,Genes Overall,tRNA Genes
0,0,2009,1,Massachusetts,Food,3,2.0,0.0,2009 1 Massachusetts,O157:H7,...,7800,0.96,0.905,39.0,25.0,5.59,50.45,"5,155","5,417",103
1,1,2010,7,Massachusetts,Other,2,2.0,0.0,2010 7 Massachusetts,O157:H7,...,7800,0.96,0.905,80.0,65.0,5.59,50.45,"5,155","5,417",103
2,2,2011,7,Massachusetts,Other,8,3.0,0.0,2011 7 Massachusetts,O157:H7,...,7800,0.96,0.905,80.0,65.0,5.59,50.45,"5,155","5,417",103
3,6,2012,4,Massachusetts,Food,2,1.0,1.0,2012 4 Massachusetts,O157:H7,...,7800,0.96,0.905,66.0,50.0,5.59,50.45,"5,155","5,417",103
4,7,2012,5,Massachusetts,Food,6,2.0,0.0,2012 5 Massachusetts,O157:H7,...,7800,0.96,0.905,76.0,60.0,5.59,50.45,"5,155","5,417",103


In [14]:
#save to CSV
all_terms.to_csv(r'all_terms.csv', index=True, header=True)